### Use LWE for encryption and decryption

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
n = 8 # security parameters / vector dimension
q = 127 # modulus
N=int(1.1*n*np.log(q)) # number of samples
sigma = 1.0
print(f"n={n},q={q},N={N},sigma={sigma}")

n=8,q=127,N=42,sigma=1.0


Now, let us introduce errors using a gaussian distribution

In [9]:
def chi(stdev, modulus):
    return round((np.random.randn() * stdev**2)) % modulus

sd = 2
m = 1000
for x in range(10):
    print("chi = ", chi(sd, m))

chi =  996
chi =  995
chi =  0
chi =  992
chi =  999
chi =  999
chi =  1
chi =  8
chi =  997
chi =  994


In [10]:
alice_private_key = np.random.randint(0, high =q, size=n)
print(f"alice's private key is {alice_private_key}")

alice's private key is [ 95 117  28 113  93 121  68  40]


In [11]:
alice_public_key =[]
for i in range(N):
    a = np.random.randint(0, high=q, size= n)
    epsilon = chi(sigma, q)
    b = (np.dot(a, alice_private_key) + epsilon) % q
    sample = (a,b)
    alice_public_key.append(sample)

print("alice public key is: ", alice_public_key)

alice public key is:  [(array([101,  98,  13,  62,   0,  29,  95, 114]), 34), (array([98, 65, 22, 27,  7, 29, 95, 95]), 79), (array([ 78,   0,  59,  33, 109, 113,  99,  32]), 38), (array([49, 58, 70, 59, 45, 43, 30, 27]), 64), (array([ 93,  74,   6, 125, 100,  44,  38,  70]), 104), (array([ 10,  20,  98,  32,   0, 123,  23, 110]), 16), (array([34, 52, 83, 79, 91, 88, 93, 39]), 61), (array([39, 67, 90, 31, 57, 75,  1, 59]), 82), (array([ 31,  53, 121,  30, 124,  41, 109,   7]), 104), (array([119,  31,  78,  26, 112,  25,  13, 101]), 66), (array([ 62,  83, 119,  81, 103,  75,  97,  24]), 68), (array([  7,  77,  80,  79,   1,  41,  27, 126]), 6), (array([ 80,  82,  55,  30,  74, 108, 123, 107]), 108), (array([86, 65, 50, 37, 26, 24, 67, 81]), 57), (array([ 94, 113,  29, 126,  28,  24,  90, 114]), 49), (array([ 79,  86,  70, 102,  99,  56,  43,  23]), 80), (array([116,  75,  39, 108,  53,  65,  69,  69]), 123), (array([  8, 100,  47,  92,  30,  72,  92,  62]), 87), (array([89, 48, 77, 54, 

In [12]:
bob_message_bit = 1
print(f"bob's message bit = {bob_message_bit}")

bob's message bit = 1


In [13]:
r = np.random.randint(0, 2, N)
print(r)

[1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1
 1 1 1 0 0]


In [16]:
sum_ai=np.zeros(n, dtype=int)
sum_bi=0

for i in range(N):
    sum_ai = sum_ai + r[i] * alice_public_key[i][0]
    sum_bi += r[i] * alice_public_key[i][1]
sum_ai = [x % q for x in sum_ai]

ciphertext = (sum_ai, (bob_message_bit * int(np.floor(q/2)) + sum_bi) %q)
print(f"ciphertext is: {ciphertext}")

ciphertext is: ([116, 38, 51, 114, 123, 99, 21, 22], 69)


In [17]:
adots = np.dot(ciphertext[0],alice_private_key) % q
b_adots = (ciphertext[1] - adots) % q

decrypted_msg_bit = round((2*b_adots)/q) %2

print(f"original message = {bob_message_bit}, decrypted message = {decrypted_msg_bit}")

assert(bob_message_bit == decrypted_msg_bit)

original message = 1, decrypted message = 1


In [18]:
bob_message_bits = np.random.randint(0, 2, 16)
print(f"Bob's message bits are : {bob_message_bits}")
decrypted_bits = []

for ib in range(len(bob_message_bits)):
    bob_message_bit = bob_message_bits[ib]

    r = np.random.randint(0, 2, N)
    
    sum_ai=np.zeros(n, dtype=int)
    sum_bi=0
    for i in range(N):
        sum_ai = sum_ai + r[i] * alice_public_key[i][0]
        sum_bi = sum_bi + r[i] * alice_public_key[i][1]
    sum_ai = [ x % q for x in sum_ai ]

    ciphertext = (sum_ai, (bob_message_bit*int(np.floor(q/2))+sum_bi)%q)

    adots = np.dot(ciphertext[0], alice_private_key) % q
    b_adots = (ciphertext[1] - adots) % q

    decrypted_message_bit = round((2*b_adots)/q) % 2
    assert decrypted_message_bit == bob_message_bit

    decrypted_bits.append(decrypted_message_bit)
    
print(f"Decrypted message bits = {np.array(decrypted_bits)}")

Bob's message bits are : [1 1 0 0 0 0 1 0 1 1 1 0 1 1 0 1]
Decrypted message bits = [1 1 0 0 0 0 1 0 1 1 1 0 1 1 0 1]
